# Python Inventory System - Session 2: Enhanced Database Operations

This notebook covers the new features added to the database operations in Session 2 of our Inventory Management System. We'll explore the enhanced functionality including search, low stock alerts, and sales tracking.

## What's New in Session 2

1. Product Search Functionality
2. Low Stock Alerts with Custom Thresholds
3. Sales Tracking System
4. Updated CRUD Operations

## 1. Product Search Functionality

We've added the ability to search products by name, description, or category using SQL `LIKE` queries.

In [ ]:
from typing import List
from sqlmodel import or_
from models import Product

def search_products(search_term: str) -> List[Product]:
    """
    Search for products by name, description, or category
    
    Args:
        search_term: Search term to look for
        
    Returns:
        List[Product]: List of matching products
    """
    search_pattern = f"%{search_term}%"
    
    with Session(engine) as session:
        statement = select(Product).where(
            or_(
                Product.name.like(search_pattern),
                Product.description.like(search_pattern),
                Product.category.like(search_pattern)
            )
        )
        products = session.exec(statement).all()
        return products

# Example usage:
# results = search_products("laptop")
# for product in results:
#     print(product.name)

### Key Features:

- Uses SQL `LIKE` operator with wildcards (`%`)
- Searches across multiple fields (name, description, category)
- Case-sensitive search (database dependent)
- Returns full Product objects with all attributes

## 2. Low Stock Alerts with Custom Thresholds

We've enhanced the product model to support customizable low stock thresholds and added a function to identify products that need restocking.

In [ ]:
def get_low_stock_products() -> List[Product]:
    """
    Get list of products with stock below their threshold
    
    Returns:
        List[Product]: List of products with low stock
    """
    with Session(engine) as session:
        statement = select(Product).where(
            Product.stock_quantity <= Product.low_stock_threshold
        )
        products = session.exec(statement).all()
        return products

# Example usage:
# low_stock_items = get_low_stock_products()
# for product in low_stock_items:
#     print(f"{product.name} (Stock: {product.stock_quantity}, Threshold: {product.low_stock_threshold})")

### Key Features:

- Each product now has a `low_stock_threshold` field
- Threshold is customizable per product (default is 10)
- Simple query to find all products needing restock
- Returns full Product objects for display or notification

## 3. Sales Tracking System

We've added comprehensive sales tracking with two new functions:

In [ ]:
from models import Sale

def record_sale(product_id: int, quantity: int, sale_price: Optional[float] = None) -> Optional[Sale]:
    """
    Record a sale of a product
    
    Args:
        product_id: ID of the product sold
        quantity: Quantity sold
        sale_price: Price per unit (if different from current product price)
        
    Returns:
        Sale: The recorded sale or None if product not found or insufficient stock
    """
    with Session(engine) as session:
        # Get the product
        product = session.get(Product, product_id)
        
        if not product:
            return None
        
        # Check stock
        if product.stock_quantity < quantity:
            return None
        
        # Create sale
        price = sale_price if sale_price is not None else product.price
        sale = Sale(
            product_id=product_id,
            quantity=quantity,
            sale_price=price
        )
        
        # Update stock
        product.stock_quantity -= quantity
        product.updated_at = datetime.now()
        
        # Save changes
        session.add(sale)
        session.add(product)
        session.commit()
        session.refresh(sale)
        
        return sale

,def get_sales_history(product_id: Optional[int] = None) -> List[Sale]:
    """
    Get sales history, optionally filtered by product
    
    Args:
        product_id: Optional product ID to filter by
        
    Returns:
        List[Sale]: List of sales
    """
    with Session(engine) as session:
        if product_id:
            statement = select(Sale).where(
                Sale.product_id == product_id
            ).order_by(Sale.sale_date.desc())
        else:
            statement = select(Sale).order_by(Sale.sale_date.desc())
        
        sales = session.exec(statement).all()
        return sales

# Example usage:
# sale = record_sale(1, 2)  # Sell 2 units of product ID 1
# history = get_sales_history(1)  # Get sales for product ID 1

### Key Features:

- **Atomic operations**: Stock is reduced only if sale is recorded
- **Flexible pricing**: Can override product price for sales/discounts
- **Stock validation**: Prevents sales that would result in negative stock
- **Comprehensive history**: View all sales or filter by product
- **Audit trail**: Each sale records date/time automatically

## 4. Updated CRUD Operations

Existing functions have been enhanced to support the new features:

In [ ]:
def add_product(
    name: str, 
    description: str, 
    price: float, 
    stock_quantity: int, 
    category: str, 
    low_stock_threshold: int = 10
) -> Product:
    """
    Add a new product with low stock threshold support
    """
    product = Product(
        name=name,
        description=description,
        price=price,
        stock_quantity=stock_quantity,
        category=category,
        low_stock_threshold=low_stock_threshold
    )
    session.add(product)
    session.commit()
    return product

def edit_product(
    product_id: int,
    low_stock_threshold: Optional[int] = None,
    **kwargs
) -> bool:
    """
    Edit product now supports low_stock_threshold updates
    """
    product = session.get(Product, product_id)
    if not product:
        return False
    
    if low_stock_threshold is not None:
        product.low_stock_threshold = low_stock_threshold
    
    # Handle other fields...
    product.updated_at = datetime.now()
    session.commit()
    return True

### Key Updates:

- `add_product` now accepts `low_stock_threshold` parameter
- `edit_product` can update the threshold
- All functions maintain consistent error handling
- Automatic timestamp updates on modifications

## Summary

In this session, we've enhanced our inventory system with:

1. **Powerful search** to quickly find products
2. **Smart inventory alerts** with customizable thresholds
3. **Comprehensive sales tracking** with history
4. **Updated operations** to support new features

These improvements make the system more practical for real-world inventory management while maintaining clean, type-safe code.